# This notebook show my issue with the calculation time of the enrichment function

In [15]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [16]:
import greatpy as great
import pandas as pd

In [17]:
test = pd.read_csv("../data/human/test_genomic_region.bed",sep="\t",comment="#",
                names=["Chr", "Chr_Start", "Chr_End"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64"})

df_10_row = test[:30]

In [18]:
%lprun -f great.tl.enrichment great.tl.enrichment(df_10_row,"../data/human/regulatory_domain.bed","../data/human/chr_size.bed","../data/human/ontologies.csv",True,True,0.05,(0,0))

Timer unit: 1e-06 s

Total time: 11.1606 s
File: /home/tom/.local/lib/python3.8/site-packages/greatpy/tl/basic.py
Function: enrichment at line 312

Line #      Hits         Time  Per Hit   % Time  Line Contents
   312                                           def enrichment(test:str or pd.DataFrame,regdom_file,chr_size_file,annotation,binom=True,hypergeom=True,alpha=0.05,correction=("fdr",0.05),sort_by=None): 
   313                                               # Data import 
   314         1        223.0    223.0      0.0      if not binom and not hypergeom : 
   315                                                   return False
   316                                               
   317         2     191539.0  95769.5      1.7      regdom = pd.read_csv(regdom_file,sep="\t",comment="#",
   318         1         68.0     68.0      0.0                      names=["Chr", "Chr_Start", "Chr_End","Name","tss","Strand"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64","Name":"

Here we can see that the calculation time of the enrichment function is very high due to : 

line 371 (50 %) in the https://github.com/theislab/greatpy/blob/correction/greatpy/tl/basic.py with hit[i] = number_of_hit(test,curr_regdom) => The number_of_hit function is used to calculate the number of genomic region hit with each GO term 

In [19]:
regdom = pd.read_csv("../data/human/regulatory_domain.bed",sep="\t",comment="#",
                    names=["Chr", "Chr_Start", "Chr_End","Name","tss","Strand"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64","Name":"object","tss":"int64","Strand":"object"})

In [23]:
def number_of_hit(test,regdom): 
    nb=0
    regdom = regdom[["Chr","Chr_Start","Chr_End"]]
    for i in range(test.shape[0]): 
        chrom = test.iat[i,0]
        start = test.iat[i,1]
        end = test.iat[i,2]
        regdom_reduce = regdom[regdom["Chr"].isin([chrom])]
        if regdom_reduce[(regdom_reduce["Chr_Start"] <= start) & (regdom_reduce["Chr_End"] >= end)].shape[0] > 0 : 
            nb += 1
    return nb
number_of_hit(test,regdom)

84

In [29]:
%lprun -f number_of_hit number_of_hit(test,regdom)

Timer unit: 1e-06 s

Total time: 0.402752 s
File: /tmp/ipykernel_7575/418560114.py
Function: number_of_hit at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def number_of_hit(test,regdom): 
     2         1         57.0     57.0      0.0      nb=0
     3         1      15322.0  15322.0      3.8      regdom = regdom[["Chr","Chr_Start","Chr_End"]]
     4       105        235.0      2.2      0.1      for i in range(test.shape[0]): 
     5       104      11083.0    106.6      2.8          chrom = test.iat[i,0]
     6       104       6768.0     65.1      1.7          start = test.iat[i,1]
     7       104       5889.0     56.6      1.5          end = test.iat[i,2]
     8       104     179931.0   1730.1     44.7          regdom_reduce = regdom[regdom["Chr"].isin([chrom])]
     9       104     183249.0   1762.0     45.5          if regdom_reduce[(regdom_reduce["Chr_Start"] <= start) & (regdom_reduce["Chr_End"] >= end)].s